## Лабораторная работа 1: Составить граф информационно-аналитических веб-ресурсов, посвященных металлургической промышленности в Российской Федерации и ближнем зарубежье.
---
Задание:

Необходимо составить список ссылок, содержащих текстовую информацию о металлургических предприятиях, процессах, происходящих в металлургической промышленности, особенностях технологических процессов, особенностях инструментов и агрегатов, применяющихся на предприятиях, особенностях функционирования предприятий.

Итогом должен стать граф на языке DOT, описывающий связи между ресурсами.

К каждому ресурсу должен быть сохранен список ключевых терминов (понятий), которые были размещены на этом ресурсе. Эти понятия должны стать основой для связи сайтов между собой

Например:

"сайт 1" -> "сайт 2"

Сайт 1: черная металлургия — это....

Сайт 2: Черные металлы – железо и его сплавы, которые отличаются от остальных металлов, называемых цветными...

Размер графа — от 25 ресурсов

Для этого понадобится:

- requests
- bs4
- graphviz (для просмотра и отладки графа)
- возможно, rutermextract (для поиска ключевых слов по их нормальной форме)
---

In [1]:
import requests
import bs4
import graphviz
from bs4 import BeautifulSoup
from itertools import combinations
import networkx 

urls = [
    
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%8F', 'keywords': ['Металлургия', 'Разновидности металлургии', 'Распространение и сферы применения', 'Сплавы и их применение', 'История', 'Добывающая металлургия', 'Чёрная металлурги', 'Доменное производство чугуна', 'Производство стали', 'Кислородно-конвертерный процесс', 'Мартеновский процесс', 'Электросталеплавильное производство', 'Легирование стали',  'Порошковая металлургия', 'Цветная металлургия', 'Производство меди', 'Производство алюминия', 'Производство других цветных металлов'] },
    {'url': 'https://ibprom.ru/metalurgicheskie_predpriyatiya', 'keywords': ['Список металлургических предприятий']},
    {'url': 'https://aitcom.ru/stati/218-metallurgiya-otrasl-promyishlennosti', 'keywords': ['Металлургия', 'Разновидности металлургии', 'Чёрная металлургия', 'Цветная металлургия', 'История', 'Добывающая металлургия', 'Свойства металлов', 'Применения металлов', 'Сплавы']},
    {'url': 'https://www.metobr-expo.ru/ru/ui/17159/', 'keywords': ['Металлургическая промышленность', 'История', 'Особенности', 'Металлургическая промышленность в России', 'Металлургические компании и предприятия', ]},
    {'url': 'https://vitapro.ru/otrasli/metallurgicheskaya-promyshlennost/', 'keywords': ['Металлургическая промышленность', 'Вредные физические факторы', 'Опасные физические факторы']},  
    {'url': 'https://elebia.com/ru/%D0%BC%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F-%D0%BF%D1%80%D0%BE%D0%BC%D1%8B%D1%88%D0%BB%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D1%8C/', 'keywords': ['Металлургическая промышленность', 'Безопасность']},  
    {'url': 'https://www.vedomosti.ru/partner/articles/2023/03/22/967340-metallurgicheskaya-otrasl', 'keywords': ['2022', 'Родные трубы', 'Взгляд в будущее', '2023']},  
    {'url': 'https://петербург-заводской.рф/metallurgicheskajapromyshlennost/', 'keywords': ['Металлургическая промышленность', 'История']},  
    {'url': 'https://cae-expert.ru/industry/metallurgicheskaya-promyshlennost', 'keywords': ['Металлургическая промышленность', 'Задачи и способы решения', 'Устойчивое развитие', 'Инновации и качество', 'Энергосбережение', 'Экономическая эффективность', 'Возможности']},  
    {'url': 'https://nc-t.ru/category-references/metallurgicheskaya-promyshlennost/', 'keywords': ['Металлургическая промышленность', 'Список металлургических предприятий']},    
    {'url': 'https://www.metaltorg.ru/analytics/color/?id=87', 'keywords': ['Цветная металлургия', 'История', 'Статистика']},   
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8', 'keywords': ['Металлургия', 'Россия', 'Чёрная металлургия', 'Трубная промышленность', 'Цветная металлургия', 'Особенности', 'Статистика', 'Санкции', '2022']},
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%8F_%D0%A1%D0%A1%D0%A1%D0%A0', 'keywords': ['Металлургия', 'СССР', 'Статистика', 'История', 'Запасы железной руды', 'Чёрная металлургия', 'Цветная металлургия']},
    {'url': 'https://www.pwc.com/kz/en/publications/metals_mining_russia_the_cis_rus_2011_fin.pdf', 'keywords': ['Россия', 'СНГ', 'Металлургия', 'Перспективы', 'Статистика']},
    {'url': 'https://cyberleninka.ru/article/n/pokazateli-chernoy-metallurgii-stran-sng-i-byvshego-sssr-do-2000-g', 'keywords': ['Чёрная металлургия', 'СНГ', 'СССР']},  
    {'url': 'https://www.karty.ru/metallurgicheskaya-promyshlennost-sng-i-gosudarstv-evropy-6mln.html', 'keywords': ['Металлургическая промышленность', 'СНГ']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81', 'keywords': ['Металлургический процесс', 'Чёрная металлургия', 'Цветная металлургия', 'Обогащение руды']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%BE%D0%BC%D0%B1%D0%B8%D0%BD%D0%B0%D1%82', 'keywords': ['Метуллургия', 'Металлургический комбинат', 'Структура', 'Производственный процесс', 'История', 'Проблемы экологии']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B1%D1%80%D0%B8%D0%BA%D0%B0', 'keywords': ['Метуллургия', 'Фабрика', 'Завод']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%A7%D1%91%D1%80%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%8F', 'keywords': ['Чёрная металлургия', 'История', 'Россия']},    
    {'url': 'https://ru.wikipedia.org/wiki/%D0%A6%D0%B2%D0%B5%D1%82%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B5%D1%82%D0%B0%D0%BB%D0%BB%D1%83%D1%80%D0%B3%D0%B8%D1%8F', 'keywords': ['Цветная металлургия', 'Тяжёлые металлы', 'Лёгкие металлы']}, 
    {'url': 'https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%BC%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D1%80%D0%B0%D0%B9', 'keywords': ['Метуллургия', 'Пермский край', 'История']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%97%D0%B5%D0%B9%D0%B3%D0%B5%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5', 'keywords': ['Метуллургия', 'Зейгерование']},  
    {'url': 'https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D1%8E%D0%BC%D0%B8%D0%BD%D0%B8%D0%B5%D0%B2%D0%B0%D1%8F_%D0%BF%D1%80%D0%BE%D0%BC%D1%8B%D1%88%D0%BB%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D1%8C', 'keywords': ['Алюминиевая промышленность', 'Цветная металлургия', 'История']},  
    {'url': 'https://www.snta.ru/press-center/chto-takoe-metallurgiya/', 'keywords': ['Метуллургия', 'История', 'Технологии','Области']},    
    {'url': 'https://resursmsk.ru/metallurgiya', 'keywords': ['Метуллургия', 'Технологический процесс', 'Разновидности', 'Сфера применения', 'Понятие']}, 
]


graf = networkx.Graph()

for i in urls:
    url = i["url"]
    keywords = i["keywords"]

    response = requests.get(url)                          
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()   
    
    graf.add_node(url)   
   
    for key in keywords:                              
        graf.nodes[url][key] = True
        

In [ ]:
for A, B in combinations(urls, 2):                    
    Awords = set(A["keywords"])
    Bwords = set(B["keywords"])
    connection = Awords.intersection(Bwords)
    
    if connection:
        graf.add_edge(A["url"], B["url"], common_keywords=list(connection)) 
   

In [ ]:
grafD = graphviz.Digraph(comment='Металлургия')  
grafD.render('/home/kirill/projects/Web_Data_Mining/Web-Data-Mining/Labs/lab_1/graph', view=False)     
    
for vertex in graf.nodes:       
    grafD.node(vertex)
    
for edge in graf.edges:
    A, B = edge
    common_keywords = graf.edges[A, B]["common_keywords"]
    record = ", ".join(common_keywords)
    grafD.edge(A, B, label=record)
               
grafD.render('Graph', view=True)  

'Граф.pdf'


(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'EvView': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'GtkScrolledWindow': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'GtkBox': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'GtkPaned': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'GtkBox': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:39.387: drawing failure for widget 'EvWindow': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:40.735: drawing failure for widget 'EvView': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:18:40.735: drawing failure for widget 'GtkScrolledWindow': invalid matrix (not invertible)

(xreader:40209): Gtk-WARNING **: 00:1